# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [3]:
# import re

# def extract_characters_with_underscore(input_string):
#     pattern = r'\b(\w)(?=_*)'
#     characters = re.findall(pattern, input_string)
#     return list(set(characters))

# # Example usage:
# test_string1 = '_ _ _ _ _ _ _ _ _ _ e'
# result1 = extract_characters_with_underscore(test_string1)
# print(result1)  # Output: ['i', 'a']

# test_string2 = 'c _ e r n o _ i t _ '

# result2 = extract_characters_with_underscore(test_string2)
# print(result2)  # Output: ['i', 'a']


def extract_characters_with_underscore(input_string):
    bigrams = input_string.split()
    char_underscore_bigrams = []

    for i in range(len(bigrams) - 1):
        if bigrams[i + 1] == '_':
            char_underscore_bigrams.append(bigrams[i])
    char_underscore_bigrams = list(set(char_underscore_bigrams))
    if '_' in char_underscore_bigrams:
        char_underscore_bigrams.remove('_')
    return char_underscore_bigrams

# Example usage:
input_string = 'n i _ j _ _ i _'
result = extract_characters_with_underscore(input_string)
print(result)  # Output: ['i', 'j']


['i', 'j']


In [4]:
import string
import pandas as pd

def create_guess_data_frame(guessed_letters):
    all_chars = string.ascii_lowercase
    data = {char: [0 if char in guessed_letters else 1 if char != guessed_letters else 0 for _ in range(len(all_chars))] for char in all_chars}
    df = pd.DataFrame(data, index=list(all_chars))
    return df.T

# Example usage:
guessed_letters = ['r', 'e']
guess_data_frame = create_guess_data_frame(guessed_letters)

print(guess_data_frame.to_numpy())


[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 

In [13]:
bigram_candidates = extract_characters_with_underscore('w o _ r _ d _')

In [14]:
# word

In [15]:
bigram_candidates

['r', 'o', 'd']

In [16]:
def build_dictionary(dictionary_file_location):
    text_file = open(dictionary_file_location,"r")
    full_dictionary = text_file.read().splitlines()
    text_file.close()
    return full_dictionary

word_list = build_dictionary('./words_250000_train.txt')
import string
import pandas as pd

def create_bigram_data_frame(word_list):
    # Step 1: Count bigrams for each character
    bigram_counts = {}
    
    for word in word_list:
        word = word.lower()  # Convert all words to lowercase for case-insensitive comparison
        word_length = len(word)
        
        for i in range(word_length - 1):
            bigram = word[i:i+2]
            
            # Count bigrams
            first_char = bigram[0]
            second_char = bigram[1]
            if first_char not in bigram_counts:
                bigram_counts[first_char] = {}
            if second_char in bigram_counts[first_char]:
                bigram_counts[first_char][second_char] += 1
            else:
                bigram_counts[first_char][second_char] = 1

    # Step 2: Create the data frame with all bigrams having count 0
    all_chars = string.ascii_lowercase
    df_data = {first_char: {second_char: bigram_counts.get(first_char, {}).get(second_char, 0) for second_char in all_chars}
               for first_char in all_chars}
    
    df = pd.DataFrame(df_data)

    return df

# Example usage:
# word_list = ['appetizingly', 'appraisingly', 'backbitingly']
bigram_data_frame = create_bigram_data_frame(word_list)

print(bigram_data_frame.T)


       a     b      c      d      e     f      g      h      i    j  ...    q  \
a    270  7258  10476   6110   3350  1383   4914   1089   4512  211  ...  204   
b   5416  1076    221    250   5974    89     48    126   5052  148  ...    5   
c  13089    28   1423     68   9023    34     32  12023   6429    1  ...  113   
d   5429   369    256   1321  13954   321    734    383  10976  175  ...    8   
e  10223  2171   7706  23098   5426  2340   2617   1014   2838  265  ...  526   
f   2623    90    197    159   3328  2055     46     61   4404   20  ...    6   
g   4660   173     66    117   7326   103   1315   2377   4788   21  ...    2   
h   9559   269    147    126  11707   202     67    109   9035   15  ...   13   
i  11742  2691  20223   8250   8301  3508   4548    274    421  106  ...  284   
j    992     2     11      8    735     3      3      5    267    6  ...    0   
k   1749   205     79     60   5188   161     38    359   3100   23  ...    3   
l  15234   669    923   2118

In [17]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link
    import collections
    import re

# class GuessingGame:
#     def __init__(self):
#         # Initialize an empty list for the current dictionary of words
#         self.current_dictionary = []
#         # Initialize an empty set to keep track of guessed letters
#         self.guessed_letters = set()

#     def load_dictionary(self, words):
#         # Load the dictionary of words into the current_dictionary list
#         self.current_dictionary = words

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        
        def contains_non_dot_characters(input_string):
            for char in input_string:
                if char != '.':
                    return True
            return False
        
        bigram_candidates = extract_characters_with_underscore(word)
                   #removed print

        # print('bigram_candidates: ', bigram_candidates)
        if len(bigram_candidates)>0:
            terminal_char_only = False
        else:
                       #removed print

            # print('Setting terminal_char_only to True')
            terminal_char_only = True
            
    
        # terminal_char_only = False
        if contains_non_dot_characters(clean_word) and terminal_char_only == False:
            #.....
            bigram_candidates = extract_characters_with_underscore(word)
                       #removed print

            # print('bigram_candidates: ', bigram_candidates)
            # if len(bigram_candidates)>0:
            #     terminal_char_only = False
                # bigram_candidates.remove('_')
                
    
            guess_data_frame = create_guess_data_frame(list(set(self.guessed_letters)))
            
            masked_bigram_frame = bigram_data_frame*guess_data_frame.to_numpy()
            all_bigram_count = masked_bigram_frame.sum().sum()
            prob_bigram_distr = masked_bigram_frame/all_bigram_count

            probability_dict = dict()
            for cand_ in bigram_candidates:
                idx_max_prob_a = prob_bigram_distr[cand_].argmax()
                p_a = prob_bigram_distr[cand_].iloc[idx_max_prob_a]
                probability_dict[cand_] = p_a

            # Get the key corresponding to the max value
            max_key = max(probability_dict, key=lambda k: probability_dict[k])
                       #removed print

            # print('Initial char of the bigram: ', max_key)  # Output: 'banana'
            idx_max_prob = prob_bigram_distr[max_key].argmax()
            
            guess_letter_a = prob_bigram_distr.columns[idx_max_prob]
                       #removed print

            # print('bigram model predicted guess word ' , guess_letter_a)    
            return guess_letter_a
            # else:
            #     print('Setting terminal_char_only to True')
            #     terminal_char_only = True
        # elif terminal_char_only == True or not contains_non_dot_characters(clean_word):
        else:
            # if terminal_char_only == True:
                           #removed print

                # print('In this loop because terminal char arrived')
            # find length of passed word
            terminal_char_only = False
            len_word = len(clean_word)
    
            
            # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
            current_dictionary = self.current_dictionary
            new_dictionary = []
            
            # iterate through all of the words in the old plausible dictionary
            for dict_word in current_dictionary:
                # continue if the word is not of the appropriate length
                if len(dict_word) != len_word:
                    continue
                    
                # if dictionary word is a possible match then add it to the current dictionary
                if re.match(clean_word,dict_word):
                    new_dictionary.append(dict_word)
                # new_dictionary.append(dict_word)
                       #removed print

            # print(new_dictionary[:3],len(new_dictionary))
            
            for wrong_guessed_letter in self.wrong_guess:
                new_dictionary = [word for word in new_dictionary if wrong_guessed_letter not in word]
           #removed print
            # print(new_dictionary[:3],len(new_dictionary))
            # overwrite old possible words dictionary with updated version
            # self.current_dictionary = new_dictionary
            
            
            # count occurrence of all characters in possible word matches
            full_dict_string = "".join(new_dictionary)
            
            c = collections.Counter(full_dict_string)
            sorted_letter_count = c.most_common()                   
            
            guess_letter = '!'
            
            # return most frequently occurring letter in all possible words that hasn't been guessed yet
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break
                
            # if no word matches in training dictionary, default back to ordering of full dictionary
            if guess_letter == '!':
                sorted_letter_count = self.full_dictionary_common_letter_sorted
                for letter,instance_count in sorted_letter_count:
                    if letter not in self.guessed_letters:
                        guess_letter = letter
                        break            
            
            return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        self.wrong_guess = []                 
        response = self.request("/new_game", {"practice":practice})
        mistake_counter = 0
        max_allowed_mistakes = 6
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')

            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                mistake_counter = max_allowed_mistakes - tries_remains
                # print(mistake_counter)
                if mistake_counter>0:
                    self.wrong_guess.append(guess_letter)
                    max_allowed_mistakes = max_allowed_mistakes - 1

                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [18]:
api = HangmanAPI(access_token="812c831aa61d4f08f8b5b9fa7b7d57", timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [19]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 94e322d67339. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '94e322d67339', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ _ _ '}
Guessing letter: i
Sever response: {'game_id': '94e322d67339', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ i _ _ '}
Guessing letter: n
Sever response: {'game_id': '94e322d67339', 'status': 'ongoing', 'tries_remains': 5, 'word': 'n _ n _ _ _ _ _ _ i _ _ '}
Guessing letter: s
Sever response: {'game_id': '94e322d67339', 'status': 'ongoing', 'tries_remains': 4, 'word': 'n _ n _ _ _ _ _ _ i _ _ '}
Guessing letter: c
Sever response: {'game_id': '94e322d67339', 'status': 'ongoing', 'tries_remains': 4, 'word': 'n _ n _ c _ _ _ _ i _ _ '}
Guessing letter: g
Sever response: {'game_id': '94e322d67339', 'status': 'ongoing', 'tries_remains': 3, 'word': 'n _ n _ c _ _ _ _ i _ _ '}
Guessing letter: t
Sever response: {'game_id': '9

In [22]:
for i in range(50):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Successfully start a new game! Game ID: 51ee5889f572. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '51ee5889f572', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ _ _ e _ '}
Guessing letter: r
Sever response: {'game_id': '51ee5889f572', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ r _ _ _ _ _ _ _ e _ '}
Guessing letter: s
Sever response: {'game_id': '51ee5889f572', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ r _ _ _ _ _ _ _ e s '}
Guessing letter: i
Sever response: {'game_id': '51ee5889f572', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ r i _ _ _ _ _ _ e s '}
Guessing letter: n
Sever response: {'game_id': '51ee5889f572', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ r i n _ _ _ n _ e s '}
Guessing letter: g
Sever response: {'game_id': '51ee5889f572', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ r i n _ _ _ n _ e s '}
Guessing letter

In [25]:
api.guessed_letters

['e', 'i', 'n', 's', 't', 'h', 'u', 'a', 'l', 'r', 'c', 'o', 'p', 'd']

In [ ]:
# extract_characters_with_underscore(word)

In [ ]:
# word = 'c _ e r n o _ i t _ '
# guessed_letters_offline = ['e', 'r', 'i']
# bigram_model(word, guessed_letters_offline)

In [ ]:
# api.guessed_letters

In [ ]:
# prob_bigram_distr['i']

In [ ]:
# prob_bigram_distr['i'].argmax()

In [ ]:
# ttt = extract_characters_with_underscore('_ _ _ _ _ _ _ _ _ _ _ _ _ e _ _ _ ')

In [ ]:
# ttt.remove('_')

In [ ]:
# ttt

In [21]:
# api.wrong_guess

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

In [ ]:
len(word_list)

In [ ]:
bigram_data_frame['a'].argmax()

In [ ]:
print(bigram_data_frame['a'])

In [ ]:
masked_bigram_frame = bigram_data_frame*guess_data_frame.to_numpy()

In [ ]:
bigram_data_frame['a']

In [ ]:
masked_bigram_frame['a']

In [ ]:
all_bigram_count = masked_bigram_frame.sum().sum()

In [ ]:
# prob_bigram_distr = bigram_data_frame/bigram_data_frame.sum(axis=0)
prob_bigram_distr = masked_bigram_frame/all_bigram_count

In [ ]:
idx_max_prob = prob_bigram_distr['a'].argmax()

In [ ]:
prob_bigram_distr['a'].iloc[idx_max_prob]

In [ ]:
prob_bigram_distr.columns[idx_max_prob]

In [ ]:
import string
import pandas as pd

def create_guess_data_frame(guessed_letters):
    all_chars = string.ascii_lowercase
    data = {char: [0 if char in guessed_letters else 1 if char != guessed_letters else 0 for _ in range(len(all_chars))] for char in all_chars}
    df = pd.DataFrame(data, index=list(all_chars))
    return df.T

# Example usage:
guessed_letters = ['r', 'e']
guess_data_frame = create_guess_data_frame(guessed_letters)

print(guess_data_frame.to_numpy())


In [ ]:
import re

def extract_characters_with_underscore(input_string):
    pattern = r'\b(\w)(?=_*)'
    characters = re.findall(pattern, input_string)
    return list(set(characters))

# Example usage:
test_string1 = '_ _ _ _ _ _ _ _ _ _ e'
result1 = extract_characters_with_underscore(test_string1)
print(result1)  # Output: ['i', 'a']

test_string2 = 'c _ e r n o _ i t _ '

result2 = extract_characters_with_underscore(test_string2)
print(result2)  # Output: ['i', 'a']


In [ ]:
def extract_characters_with_underscore(input_string):
    bigrams = input_string.split()
    char_underscore_bigrams = []

    for i in range(len(bigrams) - 1):
        if bigrams[i + 1] == '_':
            char_underscore_bigrams.append(bigrams[i])
    char_underscore_bigrams = list(set(char_underscore_bigrams))
    if '_' in char_underscore_bigrams:
        char_underscore_bigrams.remove('_')
    return char_underscore_bigrams

# Example usage:
input_string = 'n i _ j _ _ i _'
result = extract_characters_with_underscore(test_string2)
print(result)  # Output: ['i', 'j']


In [ ]:
result1.remove('_')

In [ ]:
result1

In [ ]:
guess_data_frame = create_guess_data_frame(list(set(['i'])))


In [ ]:
guess_data_frame

In [ ]:
masked_bigram_frame = bigram_data_frame*guess_data_frame.to_numpy()


In [ ]:
masked_bigram_frame

In [ ]:
all_bigram_count = masked_bigram_frame.sum().sum()
prob_bigram_distr = masked_bigram_frame/all_bigram_count


In [ ]:
prob_bigram_distr

In [ ]:

probability_dict = dict()
for cand_ in ['i']:
    idx_max_prob_a = prob_bigram_distr[cand_].argmax()
    p_a = prob_bigram_distr[cand_].iloc[idx_max_prob_a]
    probability_dict[cand_] = p_a


In [ ]:
idx_max_prob_a

In [ ]:
prob_bigram_distr['i']

In [ ]:
probability_dict

In [ ]:
word = 'n i _ _ _ _ i _ '
guessed_letters_offline = ['i', 'n']
bigram_model(word, guessed_letters_offline)

In [ ]:
bigram_candidates

In [ ]:
def bigram_model(word, guessed_letters_offline):
    bigram_candidates = extract_characters_with_underscore(word)
    print('bigram_candidates: ', bigram_candidates)
    if len(bigram_candidates)>0:
        terminal_char_only = False
        # bigram_candidates.remove('_')
        

        guess_data_frame = create_guess_data_frame(list(set(guessed_letters_offline)))
        
        masked_bigram_frame = bigram_data_frame*guess_data_frame.to_numpy()
        all_bigram_count = masked_bigram_frame.sum().sum()
        prob_bigram_distr = masked_bigram_frame/all_bigram_count

        probability_dict = dict()
        for cand_ in bigram_candidates:
            idx_max_prob_a = prob_bigram_distr[cand_].argmax()
            p_a = prob_bigram_distr[cand_].iloc[idx_max_prob_a]
            probability_dict[cand_] = p_a

        # Get the key corresponding to the max value
        max_key = max(probability_dict, key=lambda k: probability_dict[k])
        
        print('Initial char of the bigram: ', max_key)  # Output: 'banana'
        idx_max_prob = prob_bigram_distr[max_key].argmax()
        
        guess_letter_a = prob_bigram_distr.columns[idx_max_prob]
        print('bigram model predicted guess word ' , guess_letter_a)    
                
    

In [ ]:


# Get the key corresponding to the max value
max_key = max(probability_dict, key=lambda k: probability_dict[k])

print(max_key)  # Output: 'banana'
idx_max_prob = prob_bigram_distr[max_key].argmax()

guess_letter_a = prob_bigram_distr.columns[idx_max_prob]
print('bigram model predicted guess word ' , guess_letter_a)    
return guess_letter_a